In [90]:
from splinter import Browser #sign in function
import splinter
from selenium.common.exceptions import WebDriverException

browser = Browser('chrome')
url = "https://zh-tw.facebook.com/login/"
browser.visit(url)
username =input('username:')
password =input('password:')
search_input = browser.find_by_xpath('//*[@id="email"]')[0]
search_input.fill(username)
search_input = browser.find_by_xpath('//*[@id="pass"]')[0]
search_input.fill(password)
browser.find_by_xpath('//*[@id="loginbutton"]').first.click()

try:
    search_input = browser.find_by_xpath('//*[@id="approvals_code"]')[0]
    phonecheck = input('phonecheck_code:')
    search_input.fill(phonecheck)
    browser.find_by_xpath('//*[@id="checkpointSubmitButton"]').first.click()
    browser.find_by_xpath('//*[@id="u_0_0"]/div/div[2]/ul/li[2]/div/label/strong').first.click()
    browser.find_by_xpath('//*[@id="checkpointSubmitButton"]').first.click()
except splinter.exceptions.ElementDoesNotExist:
    pass

phonecheck_code:134547


In [91]:
def get_friends_list(url_input):#連線到某個人讀取他的朋友清單
    import time 
    from bs4 import BeautifulSoup
    import re
    import requests
    success = 0 #checking connection is succeeded or not
    count = 0 #define how many times we have tried to connect
    times_of_trying = 3 #the limited number we want to try
    friends_list = []
    while (success == 0 and count<times_of_trying) : #if not success to connect try few times
        try:
            browser.visit(url_input)
            if browser.find_by_text('Edit Profile') != []:
                browser.find_by_text('Friends')[0].click()
            else:
                browser.find_by_text('Friends')[1].click()
                
            soup_for_while = BeautifulSoup(browser.html, 'lxml') 
            select_condition = soup_for_while.select('#timeline-medley > div > div.mbm._5vf.sectionHeader._4khu > div.uiHeader > div > div > h3')
            
            count_for_while = 0
            while select_condition == [] and count_for_while<50:
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                soup_for_while = BeautifulSoup(browser.html, 'lxml') 
                select_condition = soup_for_while.select('#timeline-medley > div > div.mbm._5vf.sectionHeader._4khu > div.uiHeader > div > div > h3')
                time.sleep(0.2)
                count_for_while += 1

                
            success = 1 #if succeeded define success connection
            
            soup = BeautifulSoup(browser.html, 'lxml') 
            section_friends = soup.select('#collection_wrapper_2356318349')[0].select('div')[0].select('ul')
            
#             if count>5 :
            for friends in section_friends: #find the friends list and save it to friend_list
                for friend in friends.select('li'):
                    if len(friend.select('div > div > div > div > div > div > a')) != 0:
                        if friend.select('div > div > div > div > div > div > a')[0].text == 'FriendFriends':
                            friends_list.append(friend.select('div > div > div > div > div > div > a')[1]['href'].replace('?fref=pb&hc_location=friends_tab',''))
#                             print(friend.select('div > div > div > div > div > div > a')[1].text)
#                             print(friend.select('div > div > div > div > div > div > a')[1]['href'].replace('?fref=pb&hc_location=friends_tab',''))

                        else:
                            friends_list.append(friend.select('div > div > div > div > div > div > a')[0]['href'].replace('?fref=pb&hc_location=friends_tab',''))
#                             print(friend.select('div > div > div > div > div > div > a')[0].text)
#                             print(friend.select('div > div > div > div > div > div > a')[0]['href'].replace('?fref=pb&hc_location=friends_tab',''))
        except splinter.exceptions.ElementDoesNotExist:
            count += 1
            time.sleep(0.5)

        except requests.ConnectionError:
            success = 1  

        except (AttributeError,IndexError): #if user does not permit us to see his firends try few times
            count += 1
            time.sleep(0.5)
            if count==times_of_trying:
                pass
        
    return friends_list

In [ ]:
friendList = []

url = "https://www.facebook.com/v78203"
def read_frends_csv():
    global friendList
    with open("./data/friends.csv","r") as file:
        friendList = file.read().split()

try:        
    read_frends_csv()
    url = friendList[-1]
except (FileNotFoundError,IndexError):
    pass

try:
#     for friend in get_friends_list(url):
#         if friend not in friendList:
#             friendList.append(friend)
#     count_friendList = 0
#     for each in friendList:
#         for element in get_friends_list(each):
#             if element not in friendList:
#                 friendList.append(element)
#             if len(friendList)%10000 == 0:
#                 with open("./data/friends_copy.csv","w") as file:
#                     for f in friendList:
#                         file.write(f+"\n")
    
    for friend in get_friends_list(url):
        if friend not in friendList:
            friendList.append(friend)
    count_friendList = 0
    
    start_number = len(friendList) -1
    while True:
        each = friendList[start_number]
        for element in get_friends_list(each):
            if element not in friendList:
                friendList.append(element)
            if len(friendList)%10000 == 0:
                with open("./data/friends_copy.csv","w") as file:
                    for f in friendList:
                        file.write(f+"\n")
        start_number += 1

except (WebDriverException,KeyboardInterrupt) as e:
    to_fcsv()

def to_frends_csv():
    with open("./data/friends.csv","w") as file:
        for f in friendList:
            file.write(f+"\n")

In [20]:
def get_likes_list(url_input):#連線到某個人抓他按讚的粉絲頁
    import time 
    from bs4 import BeautifulSoup
    import re
    import requests

    success = 0  #checking connection is succeeded or not
    count = 0  #define how many times we have tried to connect
    likes_list = []
    
    while (success == 0 and count<4) :
        try:
            browser.visit(url_input) #go user's page and find his likes
            browser.find_by_text('More').first.click()
            browser.find_by_text('Likes').first.click()
            
            soup_for_while = BeautifulSoup(browser.html, 'lxml') 
            select_condition = soup_for_while.select('#timeline-medley > div > div.mbm._5vf.sectionHeader._4khu > div.uiHeader > div > div > h3')
            
            count_for_while = 0
            while select_condition == [] and count_for_while<5000:
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                soup_for_while = BeautifulSoup(browser.html, 'lxml') 
                select_condition = soup_for_while.select('#timeline-medley > div > div.mbm._5vf.sectionHeader._4khu > div.uiHeader > div > div > h3')
                time.sleep(0.2)
                count_for_while += 1
                
            success = 1
            
            soup = BeautifulSoup(browser.html, 'lxml') #find the likes list and save it to likes_list
            chose_link = 2 #some regular pattern that what we wand is at the second part of each block
            for link in soup.select('#collection_wrapper_2409997254')[0].select('div')[0].select('a'):
                if chose_link%3 == 0: #we get the second of each block
                    likes_list.append([link.text,link['href']])

                chose_link += 1

        except splinter.exceptions.ElementDoesNotExist:
            print(url_input,'\n','user do not permit likes reading','\n','or wrong url')
            success = 1
        except Exception as ee:
            print(ee)
            count += 1
            
    return likes_list

In [ ]:
import json

def read_frends_csv():
    global friendList
    with open("./data/friends.csv","r") as file:
        friendList = file.read().split()

def save_likeslist_to_csv():
    try:
        with open('./data/facebook_likes.csv', 'a+') as myfile:
            myfile.write(json_data_list+'\n') 
    except TypeError:
        pass
    
count = 0
json_data_list = []
for user_url in input_friends_list:
    try:
        data = get_likes_list(user_url)
        jdata = json.dumps(data,ensure_ascii=False)
        json_data_list.append(jdata)
        count += 1
        if count %100 == 0:
            save_likeslist_to_csv()
            json_data_list = []
    except (WebDriverException,KeyboardInterrupt):
        save_likeslist_to_csv()
        json_data_list = []


likesList = []

url = "https://www.facebook.com/v78203"


try:        
    read_frends_csv()
    url = friendList[-1]
except (FileNotFoundError,IndexError):
    pass

try:
    for friend in get_friends_list(url):
        if friend not in friendList:
            friendList.append(friend)
    count_friendList = 0
    for each in friendList:
        for element in get_friends_list(each):
            if element not in friendList:
                friendList.append(element)
            if len(friendList)%10000 == 0:
                with open("./data/friends_copy.csv","w") as file:
                    for f in friendList:
                        file.write(f+"\n")

except (WebDriverException,KeyboardInterrupt) as e:
    to_fcsv()

def to_frends_csv():
    with open("./data/friends.csv","w") as file:
        for f in friendList:
            file.write(f+"\n")

In [1]:
information = ('https://www.facebook.com',2)

In [ ]:
def creat_database_table_in_sqlite():
    import sqlite3
    conn = sqlite3.connect('./data/facebook.db')
    c = conn.cursor()
    c.execute("DROP TABLE IF EXISTS facebook_friends")
    c.execute('''CREATE TABLE facebook_friends
             (url text primary key, intex_number INTEGER)''')
    conn.commit()
    conn.close()

In [ ]:
def save_friends_to_sqlite(url,index_number):
    conn = sqlite3.connect('./data/facebook.db')
    c = conn.cursor()
    information = (url,)
    c.execute('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)
    conn.commit()

In [ ]:
def save_likes_as_csv(likes_list): #寫入檔案 input:list
    with open('./data/facebook_users.csv', 'a+') as myfile:
        jdata = json.dumps(likes_list,ensure_ascii=False)
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(jdata)

In [6]:
def save_users_as_csv(friends_list):
    import csv
    with open('./data/facebook_users.csv', 'a+') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(friends_list)

In [ ]:
def load_csv_as_list(friends_list,round_time):
    friend_list = []
    with open('./data/facebook_users.csv', 'a+') as myfile:
        lines = myfile.readlines()

In [135]:
import csv
url = 'https://www.facebook.com/v78203' #set the starting user
crawling_level = 0
ones_likes_list = [url,get_likes_list(url)]
ones_friends_list_firstlevel = [url,get_friends_list(url)]

with open('./data/facebook_likes.csv', 'a+') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(ones_likes_list)
for element in ones_friends_list_firstlevel[1]:
    url = element[1].replace('?fref=pb&hc_location=friends_tab','')
    ones_likes_list = [url,get_likes_list(url)]
    with open('./data/facebook_likes.csv', 'a+') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(ones_likes_list)

https://www.facebook.com/kicokico26 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/george.kao2 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/etina80610 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/yushihting 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/kaiwei.tseng.9028 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/profile.php?id=100000172058437&fref=pb&hc_location=friends_tab 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/pierce.lan 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/lewis.lin.7399 
 user do not apply likes reading 
 or wrong url
https://www.facebook.com/jung.chen.5074 
 user do not apply likes reading 
 or wrong url


KeyboardInterrupt: 